In [1]:
import requests
import pandas as pd
import json
from typing import Dict, List, Optional

def fetch_yahoo_gainers(offset: int = 0, count: int = 100) -> Dict:
    """
    Fetch stock gainers data from Yahoo Finance screener API
    
    Args:
        offset: Starting position for pagination
        count: Number of records to fetch (max 100 per request)
    
    Returns:
        Dictionary containing the API response
    """
    # Yahoo Finance screener API endpoint
    url = "https://query1.finance.yahoo.com/v1/finance/screener"
    
    # Headers to mimic a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/json',
        'Accept-Language': 'en-US,en;q=0.9',
        'Origin': 'https://finance.yahoo.com',
        'Referer': 'https://finance.yahoo.com/',
    }
    
    # Parameters for the gainers screener
    params = {
        'crumb': 'undefined',  # Often not strictly required
        'lang': 'en-US',
        'region': 'US',
        'formatted': 'true',
        'corsDomain': 'finance.yahoo.com'
    }
    
    # Request body for gainers screener
    # This mimics the "Day Gainers" preset screener
    payload = {
        "size": count,
        "offset": offset,
        "sortField": "percentchange",
        "sortType": "DESC",
        "quoteType": "EQUITY",
        "topOperator": "AND",
        "query": {
            "operator": "AND",
            "operands": [
                {
                    "operator": "gt",
                    "operands": ["percentchange", 0]
                },
                {
                    "operator": "eq",
                    "operands": ["region", "us"]
                },
                {
                    "operator": "or",
                    "operands": [
                        {
                            "operator": "BTWN",
                            "operands": ["intradaymarketcap", 2000000000, 10000000000]
                        },
                        {
                            "operator": "BTWN",
                            "operands": ["intradaymarketcap", 10000000000, 100000000000]
                        },
                        {
                            "operator": "gt",
                            "operands": ["intradaymarketcap", 100000000000]
                        }
                    ]
                },
                {
                    "operator": "gt",
                    "operands": ["dayvolume", 15000]
                }
            ]
        },
        "userId": "",
        "userIdType": "guid"
    }
    
    try:
        response = requests.post(url, headers=headers, params=params, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def extract_stock_data(api_response: Dict) -> List[Dict]:
    """
    Extract stock data from the API response
    
    Args:
        api_response: The JSON response from Yahoo Finance API
    
    Returns:
        List of dictionaries containing stock data
    """
    stocks = []
    
    if not api_response or 'finance' not in api_response:
        return stocks
    
    try:
        results = api_response['finance']['result'][0]['quotes']
        
        for quote in results:
            stock_data = {
                'Symbol': quote.get('symbol', 'N/A'),
                'Name': quote.get('longName', quote.get('shortName', 'N/A')),
                'Price (Intraday)': quote.get('regularMarketPrice', {}).get('raw', 0),
                'Change': quote.get('regularMarketChange', {}).get('raw', 0),
                '% Change': quote.get('regularMarketChangePercent', {}).get('raw', 0),
                'Volume': quote.get('regularMarketVolume', {}).get('raw', 0),
                'Avg Vol (3 month)': quote.get('averageDailyVolume3Month', {}).get('raw', 0),
                'Market Cap': quote.get('marketCap', {}).get('raw', 0),
                '52 Week Range': f"{quote.get('fiftyTwoWeekLow', {}).get('fmt', 'N/A')} - {quote.get('fiftyTwoWeekHigh', {}).get('fmt', 'N/A')}",
                'PE Ratio (TTM)': quote.get('trailingPE', {}).get('raw', None),
                'EPS (TTM)': quote.get('epsTrailingTwelveMonths', {}).get('raw', None),
            }
            stocks.append(stock_data)
            
    except (KeyError, IndexError) as e:
        print(f"Error extracting data: {e}")
    
    return stocks

def get_all_gainers() -> pd.DataFrame:
    """
    Fetch all gainers by paginating through the API
    
    Returns:
        DataFrame containing all gainers data
    """
    all_stocks = []
    offset = 0
    batch_size = 100  # Max allowed per request
    total_count = None
    
    print("Fetching Yahoo Finance gainers data...")
    
    while True:
        print(f"Fetching records {offset + 1} to {offset + batch_size}...")
        
        # Fetch data from API
        response = fetch_yahoo_gainers(offset=offset, count=batch_size)
        
        if not response:
            break
        
        try:
            # Get total count if not already known
            if total_count is None:
                total_count = response['finance']['result'][0]['total']
                print(f"Total gainers available: {total_count}")
            
            # Extract stock data
            stocks = extract_stock_data(response)
            
            if not stocks:
                break
                
            all_stocks.extend(stocks)
            print(f"Retrieved {len(stocks)} stocks (Total so far: {len(all_stocks)})")
            
            # Check if we've fetched all available data
            if len(all_stocks) >= total_count or len(stocks) < batch_size:
                break
            
            # Move to next batch
            offset += batch_size
            
        except (KeyError, TypeError) as e:
            print(f"Error processing response: {e}")
            break
    
    # Create DataFrame
    df = pd.DataFrame(all_stocks)
    return df

def format_large_numbers(df: pd.DataFrame) -> pd.DataFrame:
    """
    Format large numbers for better readability
    
    Args:
        df: DataFrame with numeric columns
    
    Returns:
        DataFrame with formatted columns added
    """
    df = df.copy()
    
    # Format Market Cap
    if 'Market Cap' in df.columns:
        df['Market Cap Formatted'] = df['Market Cap'].apply(lambda x: 
            f"${x/1e12:.2f}T" if x >= 1e12 else
            f"${x/1e9:.2f}B" if x >= 1e9 else
            f"${x/1e6:.2f}M" if x >= 1e6 else
            f"${x:.0f}" if pd.notna(x) else "N/A"
        )
    
    # Format Volume
    if 'Volume' in df.columns:
        df['Volume Formatted'] = df['Volume'].apply(lambda x:
            f"{x/1e6:.2f}M" if x >= 1e6 else
            f"{x/1e3:.2f}K" if x >= 1e3 else
            f"{x:.0f}" if pd.notna(x) else "N/A"
        )
    
    # Format Price
    if 'Price (Intraday)' in df.columns:
        df['Price Formatted'] = df['Price (Intraday)'].apply(lambda x: 
            f"${x:.2f}" if pd.notna(x) else "N/A"
        )
    
    # Format Change
    if 'Change' in df.columns:
        df['Change Formatted'] = df['Change'].apply(lambda x: 
            f"${x:+.2f}" if pd.notna(x) else "N/A"
        )
    
    # Format % Change
    if '% Change' in df.columns:
        df['% Change Formatted'] = df['% Change'].apply(lambda x: 
            f"{x:+.2f}%" if pd.notna(x) else "N/A"
        )
    
    return df

def save_gainers_data(df: pd.DataFrame, base_filename: str = 'yahoo_gainers') -> None:
    """
    Save the gainers data to multiple formats
    
    Args:
        df: DataFrame containing the gainers data
        base_filename: Base name for the output files
    """
    if df.empty:
        print("No data to save")
        return
    
    # Save to CSV
    csv_file = f"{base_filename}.csv"
    df.to_csv(csv_file, index=False)
    print(f"\n✓ Data saved to {csv_file}")
    
    # Save to Excel with formatting
    excel_file = f"{base_filename}.xlsx"
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Gainers', index=False)
        
        # Auto-adjust column widths
        worksheet = writer.sheets['Gainers']
        for column in worksheet.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = min(max_length + 2, 50)
            worksheet.column_dimensions[column_letter].width = adjusted_width
    
    print(f"✓ Data saved to {excel_file}")
    
    # Save to JSON
    json_file = f"{base_filename}.json"
    df.to_json(json_file, orient='records', indent=2)
    print(f"✓ Data saved to {json_file}")
    
    # Display summary statistics
    print(f"\n📊 Summary Statistics:")
    print(f"Total stocks: {len(df)}")
    print(f"Average % gain: {df['% Change'].mean():.2f}%")
    print(f"Median % gain: {df['% Change'].median():.2f}%")
    print(f"Max % gain: {df['% Change'].max():.2f}%")
    print(f"Total volume: {df['Volume'].sum():,.0f}")
    
    # Display top 10 gainers
    print(f"\n🔝 Top 10 Gainers:")
    top_10 = df.nlargest(10, '% Change')[['Symbol', 'Name', 'Price Formatted', 
                                           'Change Formatted', '% Change Formatted', 
                                           'Volume Formatted', 'Market Cap Formatted']]
    print(top_10.to_string(index=False))

# Alternative: Direct API call for predefined screener
def get_gainers_predefined() -> pd.DataFrame:
    """
    Alternative method using Yahoo's predefined gainers screener
    """
    url = "https://query1.finance.yahoo.com/v1/finance/screener/predefined/saved"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
    }
    
    params = {
        'scrIds': 'day_gainers',
        'count': 250,  # Request max count
        'corsDomain': 'finance.yahoo.com'
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Extract quotes
        quotes = data['finance']['result'][0]['quotes']
        
        # Convert to DataFrame
        stocks = []
        for quote in quotes:
            stock_data = {
                'Symbol': quote.get('symbol', 'N/A'),
                'Name': quote.get('longName', quote.get('shortName', 'N/A')),
                'Price (Intraday)': quote.get('regularMarketPrice', 0),
                'Change': quote.get('regularMarketChange', 0),
                '% Change': quote.get('regularMarketChangePercent', 0),
                'Volume': quote.get('regularMarketVolume', 0),
                'Avg Vol (3 month)': quote.get('averageDailyVolume3Month', 0),
                'Market Cap': quote.get('marketCap', 0),
                'PE Ratio (TTM)': quote.get('trailingPE', None),
            }
            stocks.append(stock_data)
        
        return pd.DataFrame(stocks)
        
    except Exception as e:
        print(f"Error with predefined screener: {e}")
        return pd.DataFrame()

# Main execution
if __name__ == "__main__":
    print("=" * 60)
    print("Yahoo Finance Daily Gainers Scraper")
    print("=" * 60)
    
    # Method 1: Use custom screener with pagination
    print("\nMethod 1: Using custom screener API with pagination...")
    df = get_all_gainers()
    
    # If Method 1 fails or returns limited data, try Method 2
    if df.empty or len(df) < 100:
        print("\nTrying Method 2: Using predefined gainers screener...")
        df = get_gainers_predefined()
    
    if not df.empty:
        # Add formatted columns
        df = format_large_numbers(df)
        
        # Sort by % Change descending
        df = df.sort_values('% Change', ascending=False)
        
        # Reset index
        df.reset_index(drop=True, inplace=True)
        
        # Save the data
        save_gainers_data(df)
    else:
        print("\n❌ Failed to retrieve data. Please check your internet connection and try again.")

Yahoo Finance Daily Gainers Scraper

Method 1: Using custom screener API with pagination...
Fetching Yahoo Finance gainers data...
Fetching records 1 to 100...
Error fetching data: 401 Client Error: Unauthorized for url: https://query1.finance.yahoo.com/v1/finance/screener?crumb=undefined&lang=en-US&region=US&formatted=true&corsDomain=finance.yahoo.com

Trying Method 2: Using predefined gainers screener...

✓ Data saved to yahoo_gainers.csv
✓ Data saved to yahoo_gainers.xlsx
✓ Data saved to yahoo_gainers.json

📊 Summary Statistics:
Total stocks: 250
Average % gain: 5.76%
Median % gain: 4.33%
Max % gain: 79.52%
Total volume: 2,952,716,174

🔝 Top 10 Gainers:
Symbol                           Name Price Formatted Change Formatted % Change Formatted Volume Formatted Market Cap Formatted
  OPEN     Opendoor Technologies Inc.          $10.52           $+4.66            +79.52%         1017.31M               $7.74B
   WBD   Warner Bros. Discovery, Inc.          $16.17           $+3.63         